In [1]:
import numpy as np
import pandas as pd
from nltk.stem.snowball import RussianStemmer
from collections import Counter
from nltk.tokenize import TweetTokenizer
import re
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical
from sklearn.model_selection import train_test_split

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
dftrain = pd.read_csv("train_task1_latest.csv", encoding='utf-8')[:1000]
dftest = pd.read_csv("sdsj_A_test.csv", encoding='utf-8')

dftrain.head()

,paragraph_id,question_id,paragraph,question,target
0,1094,46273,"В отличие от рыб, земноводные (амфибии) и прес...",С какого года Русское Царство перешло на летои...,0.0
1,7414,19164,В 1049 году Балдуину V удалось отнять у Герман...,Кто упомянул о его первых разногласиях со Штей...,0.0
2,6744,39767,Стремление достичь предельных значений ёмкости...,Как называется имеющая мировое значение эпоха ...,0.0
3,7300,36318,Первый практически пригодный двухтактный газов...,Что усугублялось из-за международного давления...,0.0
4,7077,41534,Требуя от художника углубленного изучения изоб...,Какой характер носят пророчества Леонардо да В...,0.0


In [3]:
columns = ['paragraph_id', 'question_id', 'paragraph', 'question'];

stemer = RussianStemmer()
regex = re.compile('[^а-яА-Я ]')
stem_cache = {}

def get_stem(token):
    stem = stem_cache.get(token, None)
    if stem:
        return stem
    token = regex.sub('', token).lower()
    stem = stemer.stem(token)
    stem_cache[token] = stem
    return stem

In [4]:
stem_count = Counter()
tokenizer = TweetTokenizer()

def count_unique_tokens(texts):
    for text in texts:
        tokens = tokenizer.tokenize(text)
        for token in tokens:
            stem = get_stem(token)
            stem_count[stem] += 1

count_unique_tokens(dftrain.paragraph)
count_unique_tokens(dftrain.question)

In [5]:
print("Total unique stems found: ", len(stem_count))

Total unique stems found:  10513


In [6]:
VOCAB_SIZE = 10000
vocab = sorted(stem_count, key=stem_count.get, reverse=True)[:VOCAB_SIZE]
print(vocab[:100])

['', 'в', 'и', 'с', 'на', 'год', 'как', 'был', 'котор', 'по', 'к', 'эт', 'для', 'не', 'из', 'от', 'ил', 'что', 'а', 'сво', 'друг', 'он', 'при', 'так', 'ег', 'за', 'такж', 'но', 'у', 'част', 'врем', 'то', 'одн', 'о', 'е', 'вид', 'перв', 'явля', 'во', 'их', 'же', 'до', 'может', 'нов', 'бол', 'образ', 'посл', 'межд', 'например', 'век', 'больш', 'форм', 'втор', 'банк', 'называ', 'сам', 'течен', 'прав', 'под', 'действ', 'быт', 'когд', 'чем', 'том', 'разум', 'мног', 'стал', 'со', 'времен', 'систем', 'мир', 'сем', 'отношен', 'человек', 'нескольк', 'все', 'ещ', 'тог', 'могут', 'тольк', 'некотор', 'ива', 'фильм', 'организм', 'лет', 'уж', 'ряд', 'двигател', 'через', 'сын', 'об', 'язык', 'территор', 'использова', 'опер', 'писател', 'развит', 'город', 'французск', 'возможн']


In [7]:
token_2_idx = {vocab[i] : i for i in range(VOCAB_SIZE)}

In [8]:
def text_to_vector(text, show_unknowns=False):
    vector = np.zeros(VOCAB_SIZE, dtype=np.int_)
    for token in tokenizer.tokenize(text):
        stem = get_stem(token)
        idx = token_2_idx.get(stem, None)
        if idx is not None:
            vector[idx] = 1
        elif show_unknowns:
            print("Unknown token: {}".format(token))
    return vector

In [9]:
text = dftrain.paragraph[0]
print("text: {}".format(text))
print("vector: {}".format(text_to_vector(text)[:10]))

text: В отличие от рыб, земноводные (амфибии) и пресмыкающиеся (рептилии или гады) уже имеют два круга кровообращения и сердце у них трёхкамерное (появляется межпредсердная перегородка). Единственные современные рептилии, имеющие хотя и неполноценное (межпредсердиевая перегородка не полностью разделяет предсердия, что скорей всего связано с переходом предков к полуводному образу жизни и снижению активности), но уже четырёхкамерное сердце — крокодилы. Считается, что впервые четырёхкамерное сердце появилось у примитивных архозавров и развитых синапсидов. В дальнейшем такое строение сердца унаследовали прямые потомки динозавров — птицы и потомки примитивных млекопитающих — современные млекопитающие.
vector: [1 1 1 1 0 0 0 0 0 0]


In [10]:
text_vectors = np.zeros(
    (len(dftrain.paragraph), VOCAB_SIZE*2), 
    dtype=np.int_)

for ind in range(len(dftrain.paragraph)):
    text_vectors[ind][:VOCAB_SIZE] = text_to_vector(dftrain.paragraph[ind])
    
for ind in range(len(dftrain.question)):
    text_vectors[ind][VOCAB_SIZE:] = text_to_vector(dftrain.question[ind])

In [11]:
labels = dftrain.target

In [12]:
X = text_vectors
y = to_categorical(labels, 2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [28]:
def build_model(learning_rate=0.1):
    tf.reset_default_graph()
    
    net = tflearn.input_data([None, VOCAB_SIZE*2])
    net = tflearn.fully_connected(net, 125, activation='ReLU')
    net = tflearn.fully_connected(net, 25, activation='ReLU')
    net = tflearn.fully_connected(net, 2, activation='softmax')
    regression = tflearn.regression(
        net, 
        optimizer='sgd', 
        learning_rate=learning_rate, 
        loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    return model

In [29]:
model = build_model(learning_rate=0.75)

In [31]:
model.fit(
    X_train, 
    y_train, 
    validation_set=0.1, 
    show_metric=True, 
    batch_size=128, 
    n_epoch=30)

Training Step: 164  | total loss: 0.37683 | time: 0.118s
| SGD | epoch: 033 | loss: 0.37683 - acc: 0.9100 -- iter: 512/630
Training Step: 165  | total loss: 0.35898 | time: 1.160s
| SGD | epoch: 033 | loss: 0.35898 - acc: 0.9165 | val_loss: 0.37102 - val_acc: 0.8000 -- iter: 630/630
--


In [ ]:
predictions = (np.array(model.predict(X_test))[:,0] >= 0.5).astype(np.int_)
accuracy = np.mean(predictions == y_test[:,0], axis=0)
print("Accuracy: ", accuracy)